<a href="https://colab.research.google.com/github/Falconwatch/hackaton/blob/master/Preprocessor_Realisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Реализация ядра

In [0]:
import numpy as np
class Preporcessor:
  def __init__(self, filters):
    self._filters=filters

  
  def preprocess(self, initinal_strings):
    result= list()
    #входной массив обрабатываем поэлементно
    for initinal_string in initinal_strings:
      #если вдруг есть
      result.append(self._process_string(initinal_string))
    return np.concatenate(result)

  def _process_string(self, adress_string):
    result = list()
    #сплтим по переводу строки, если есть
    substrings = [s.lower() for s in adress_string.split('\n')]
    for substring in substrings:
      for one_filter in self._filters:
        substring = one_filter(substring)
      #зздесь вызываем поочердёно фильтры, лежащие в self._filters
      result.append(substring)
   
    return result


def filter_lower(input_string):
  # тут происходит фильтрация: сплит, выкидывание, склейка. Это самый простой вариант фильтра - пустой :)
  return input_string

#переводит римские числа в арабские
def filter_roman(input_string):
  def roman_to_int(s):
    rom_val = {'i': 1, 'v': 5, 'x': 10, 'l': 50, 'c': 100, 'd': 500, 'm': 1000}
    int_val = 0
    for i in range(len(s)):
      if s[i] not in rom_val:
        return s
      if i > 0 and rom_val[s[i]] > rom_val[s[i - 1]]:
        int_val += rom_val[s[i]] - 2 * rom_val[s[i - 1]]
      else:
        int_val += rom_val[s[i]]
    return str(int_val)

  splited_string = input_string.split(' ')

  for i in range(len(splited_string)):
    splited_string[i] =roman_to_int(splited_string[i] ) 

  result = ' '.join(splited_string)
  return result

# фильтруем названия станций метро и мцк
def filter_stations(input_string):
  splited_string = input_string.split(',')
  for i in range(len(splited_string)):
    w = splited_string[i]
    if (w[:2]=='м.') or (w[-2:]==' м') or ('мцк' in w):
      splited_string[i]=''
  return ','.join(splited_string)


# убираем плохие символы из строки
def filter_bad_signs(input_string):
  input_string = input_string.replace('№','')
  input_string = input_string.replace('/','')
  return input_string

# фиксим аббревиатуры
def filter_abb(input_string):
  input_string = input_string.replace('р-он','район')
  return input_string
    
def filter_exclam(input_string):
  input_string = input_string.replace('!-', '1-')
  return input_string

In [76]:
data =['string1 example', 'string2\nкорп. ivx', 'м. войковская, дом №2', '!-я владимирская']

p = Preporcessor([filter_exclam, filter_nomer_sign, filter_roman, filter_bad_signs, filter_abb, filter_stations])
p.preprocess(data)

array(['string1 example', 'string2', 'корп. 4', ', дом 2',
       '1-я владимирская'], dtype='<U16')

# Работа

In [0]:
filters = [filter_roman, filter_stations]

In [0]:
import pandas as pd

In [79]:
bads = pd.read_parquet('bads_fixed.parquet')
bads.head()

,id,address
0,20000001,"115280, г. Москва, ул. Восточная, д. 9 этаж 1 ..."
1,20000002,"115280, г. Москва, ул. Ленинская Слобода, д. 2..."
2,20000003,"117105, г. Москва, шоссе Варшавское, д. 1 стр...."
3,20000004,"117321, г. Москва, ул. Профсоюзная, д. 128, ко..."
4,20000005,"121357, г. Москва, ул. Нежинская, д. 1 корп. 4..."


In [0]:
fixed_addresses = p.preprocess(bads['address'].values)

In [81]:
fixed_addresses

array(['115280, г. москва, ул. восточная, д. 9 этаж 1 пом. 4 ком. 1',
       '115280, г. москва, ул. ленинская слобода, д. 26 помещение 13 ком. 27',
       '117105, г. москва, шоссе варшавское, д. 1 стр. 1-2 ком. 3б', ...,
       'шоссе энтузиастов, д. 2, стр. 44, ст. москва-товарная-курская',
       'шоссе энтузиастов, д. 2, стр. 9, ст. москва-товарная-курская',
       'ывы, г.москва, аллея. жемчуговой, дом 1, корп. 1'], dtype='<U307')

In [0]:
bads.loc[0, 'address']

In [0]:
# Реализация оболочки